In [5]:
import sys
sys.path.append('/Users/sanjaydevarajan/Desktop/ML Projects/PremierLeague_TicketDemand_Predictor')



In [6]:
from src.components.data_info import *
from src.paths import *
import pandas as pd

ModuleNotFoundError: No module named 'sklearn.prepreprocessing'